## Attach Docker File

In [1]:
!cat container/Dockerfile
!asp personal_account

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Stuart Loxton <stuart.loxton@xinja.com.au>

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3 \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Install all of the packages
RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py
RUN pip3 install numpy
RUN pip3 install scipy
RUN pip3 install scikit-learn
RUN pip3 install pandas
RUN pip3 install flask
RUN pip3 install gevent
RUN pip3 install gunicorn
RUN pip3 install tensorflow
RUN pip3 install keras
RUN (cd /usr/local/lib/python3.6/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .)
RUN rm -rf /root/.cache

# Env Variables
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

# Set up the pro

In [7]:
%%sh

# The name of our algorithm
algorithm_name=ann-churn

cd container

chmod +x ann/train
chmod +x ann/serve

account=$(aws sts get-caller-identity --query Account --output text)
# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-ap-southeast-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" 

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
# $(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:ap-southeast-2:224041885527:repository/ann-churn",
            "registryId": "224041885527",
            "repositoryName": "ann-churn",
            "repositoryUri": "224041885527.dkr.ecr.ap-southeast-2.amazonaws.com/ann-churn",
            "createdAt": 1550486736.0
        }
    ]
}

Step 1/20 : FROM ubuntu:16.04
 ---> 7e87e2b3bf7a
Step 2/20 : MAINTAINER Stuart Loxton <stuart.loxton@xinja.com.au>
 ---> Using cache
 ---> 90c8ebf6650c
Step 3/20 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 6c34af7bb62b
Step 4/20 : RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py
 ---> Using cache
 ---> fc80ce9af1a1
Step 5/20 : RUN pip3 install numpy
 ---> Using cache
 ---> 09fb7ab0a7a5
Step 6/20 : RUN pip3 install scipy
 ---> Using cache
 ---> 7e10fcd3f4a2
Ste

The command '/bin/sh -c (cd /usr/local/lib/python3.6/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .)' returned a non-zero code: 1
Error response from daemon: No such image: ann-churn:latest
An image does not exist locally with the tag: 224041885527.dkr.ecr.ap-southeast-2.amazonaws.com/ann-churn


CalledProcessError: Command 'b'\n# The name of our algorithm\nalgorithm_name=ann-churn\n\ncd container\n\nchmod +x ann/train\nchmod +x ann/serve\n\naccount=$(aws sts get-caller-identity --query Account --output text)\n# Get the region defined in the current configuration (default to us-west-2 if none defined)\nregion=$(aws configure get region)\nregion=${region:-ap-southeast-2}\n\nfullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"\n\n# If the repository doesn\'t exist in ECR, create it.\naws ecr describe-repositories --repository-names "${algorithm_name}" \n\nif [ $? -ne 0 ]\nthen\n    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null\nfi\n\n# Get the login command from ECR and execute it directly\n# $(aws ecr get-login --region ${region} --no-include-email)\n\n# Build the docker image locally with the image name and then push it to ECR\n# with the full name.\n\ndocker build  -t ${algorithm_name} .\ndocker tag ${algorithm_name} ${fu

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [ ]:
# sagemaker_session.upload_data('data.csv', bucket='ml-sagemaker-bucket',
#                               key_prefix='train')